<a href="https://colab.research.google.com/github/bruAristimunha/miningAbandonment/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#bibliotecas empregadas nessa analise de dados
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import scipy as sci
import sklearn.preprocessing as sk # LabelEnconder

O presente conjunto de dados foi obtido conforme a [descrição](https://github.com/bruAristimunha/miningAbandonment) do projeto.


Apesar de o documento estar consistente. Há padrões de formatações indesejados para os algoritmos empregados nessa análise. Como, por exemplo, apenas informação sobre o **ID** em uma linha e as demais com valores faltantes, ou a falta de informação sobre os créditos das disciplinas.


Mesmo considerando que canal de obtenção de informação estar disponível, optamos pelo pre-processamento da informação levando em consideração o tempo de resposta.

In [2]:
link = "https://raw.githubusercontent.com/bruAristimunha/miningAbandonment/master/data/bcc.csv"
dados =  pd.read_csv(link,sep=";")
dados.head()

,ID DO ALUNO,ANO,QUADRIMESTRE,NOME_DISCIPLINA,COD_DISCIPLINA,CATEGORIA,A,B,C,D,F,O
0,1.0,2011.0,2º Quadrimestre,Bases Computacionais da Ciência,BIM0005-13,Obrigatória,NaN,NaN,NaN,1.0,NaN,NaN
1,NaN,NaN,NaN,Bases Matemáticas,BIN0003-13,Obrigatória,NaN,NaN,NaN,NaN,1.0,NaN
2,NaN,NaN,NaN,Iniciação à Pesquisa Científica I,ESZX129-13,Opção Limitada,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Origem da Vida e Diversidade dos Seres Vivos,BIL0304-13,Obrigatória,NaN,1.0,NaN,NaN,NaN,NaN
4,NaN,NaN,3º Quadrimestre,Geometria Analítica,BCN0404-13,Obrigatória,NaN,NaN,NaN,NaN,1.0,NaN


Observamos dois padrões de preenchimento distintos ao que chamamos de identificação (aluno, ano, quadrimestre) e o conceitos obtidos. 

Para tanto, realizamos a separação dos dados e para o preenchimento dos valores faltantes de formas diferentes.

In [0]:
#Inicialmente separamos os conceitos dos alunos, removendo todos as informações
#que não estão relacionadas aos conceitos.
conceitos = dados.drop(['ID DO ALUNO', 'ANO', 
                        'QUADRIMESTRE', 'NOME_DISCIPLINA', 
                        'COD_DISCIPLINA', 'CATEGORIA'],axis=1).copy()

#Preenchemos os valores faltantes com valores 0.
conceitos = conceitos.fillna(0)

#repetimos o mesmo processo com complemento dos dados
identificacao = dados.drop(['A','B','C','D','F','O'],axis=1).copy()

#Sempre que há um valor faltante se preenche com o valor na linha superior 
#da mesma coluna
identificacao = identificacao.fillna(method='ffill')

In [0]:
#Juntamos esses dois conjuntos preenchidos com regras de negócio distantas na 
#variável tratada
tratado = pd.concat([identificacao,conceitos],axis=1)

#Ao lidar com string e por consequencia comparações entre, realizamos uma série 
#de padronizações visando tratar erros de grafia, ou variações por acentuação

#Remoção de acentos
tratado['NOME_DISCIPLINA'] = tratado['NOME_DISCIPLINA'].str.normalize('NFKD')

#Para minusculas
tratado['NOME_DISCIPLINA'] = tratado['NOME_DISCIPLINA'].str.lower()

#Encondificação para ASCII ignorando erros
tratado['NOME_DISCIPLINA'] = tratado['NOME_DISCIPLINA'].str.encode('ascii', 
                                                                errors='ignore')

#Decoficação para UTF-8, formato mais empregado nessa categoria de análise
tratado['NOME_DISCIPLINA'] = tratado['NOME_DISCIPLINA'].str.decode('utf-8')



In [5]:
#Resultado após o processo de preenchimento e padronização
tratado.head()

,ID DO ALUNO,ANO,QUADRIMESTRE,NOME_DISCIPLINA,COD_DISCIPLINA,CATEGORIA,A,B,C,D,F,O
0,1.0,2011.0,2º Quadrimestre,bases computacionais da ciencia,BIM0005-13,Obrigatória,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,2011.0,2º Quadrimestre,bases matematicas,BIN0003-13,Obrigatória,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,2011.0,2º Quadrimestre,iniciacao a pesquisa cientifica i,ESZX129-13,Opção Limitada,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,2011.0,2º Quadrimestre,origem da vida e diversidade dos seres vivos,BIL0304-13,Obrigatória,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,2011.0,3º Quadrimestre,geometria analitica,BCN0404-13,Obrigatória,0.0,0.0,0.0,0.0,1.0,0.0


## Seguindo, optamos nessa etapa pela criação das seguintes variáveis.



1.  Tempo na Universidade em Anos; [OK]
2.  Quantidade de créditos por quadrimestre; [OK]
3.  Coeficiente de rendimento por quadrimestre; [OK]
4.  Coeficiente de rendimento por ano; [OK]
5.  Coeficiente de rendimento atual; [OK]
6.  Porcentagem de reprovação e abandono nas disciplinas; [OK]
7  Diferença por quadrimestre da média de créditos global menos a quantidade no quadrimestre; [OK]
8.  z-score entre a quantidade créditos cursadas; [OK]  
9.  Distância entre o quadrimestre ideal e o quadrimestre cursado; [OK]


###   1. Tempo na Universidade em Anos

In [0]:
#Junção de ano + quadrimestre
tratado['ANO_QUAD']= tratado['ANO'].astype(str)+tratado['QUADRIMESTRE']

#Quadrimestre cursado obtido pelo groupby por ALUNO, aplicando no ANO_QUAD 
#uma codificação para label através da função do Sklearn, 
#além de aplicar a concatenação desses multiplos groupbys + 1 
tratado['QUAD_CURSADO'] = np.concatenate(tratado.groupby("ID DO ALUNO")['ANO_QUAD'].apply(sk.LabelEncoder().fit_transform).values) + 1

#Obteção dos anos na universidade dado a contagem de créditos.
tratado['ANOS_NA_UNIVER'] = tratado['QUAD_CURSADO']//3

In [7]:
tratado.head(5)

,ID DO ALUNO,ANO,QUADRIMESTRE,NOME_DISCIPLINA,COD_DISCIPLINA,CATEGORIA,A,B,C,D,F,O,ANO_QUAD,QUAD_CURSADO,ANOS_NA_UNIVER
0,1.0,2011.0,2º Quadrimestre,bases computacionais da ciencia,BIM0005-13,Obrigatória,0.0,0.0,0.0,1.0,0.0,0.0,2011.02º Quadrimestre,1,0
1,1.0,2011.0,2º Quadrimestre,bases matematicas,BIN0003-13,Obrigatória,0.0,0.0,0.0,0.0,1.0,0.0,2011.02º Quadrimestre,1,0
2,1.0,2011.0,2º Quadrimestre,iniciacao a pesquisa cientifica i,ESZX129-13,Opção Limitada,1.0,0.0,0.0,0.0,0.0,0.0,2011.02º Quadrimestre,1,0
3,1.0,2011.0,2º Quadrimestre,origem da vida e diversidade dos seres vivos,BIL0304-13,Obrigatória,0.0,1.0,0.0,0.0,0.0,0.0,2011.02º Quadrimestre,1,0
4,1.0,2011.0,3º Quadrimestre,geometria analitica,BCN0404-13,Obrigatória,0.0,0.0,0.0,0.0,1.0,0.0,2011.03º Quadrimestre,2,0


## 2. Quantidade de créditos por quadrimestre;

Para essa análise necessitamos da quantidade créditos teóricos, práticos e individuais de cada disciplinas. Para tanto, buscamos o [catálogo de disciplinas](prograd.ufabc.edu.br/pdf/catalogo_disciplinas_grad>uacao_2017_2018_v2.xlsx) da universidade. Removemos as informações sobre as ementas desse arquivo anteriormente.

In [8]:
catalogo = 'https://raw.githubusercontent.com/bruAristimunha/miningAbandonment/master/data/catalogo_disciplinas_graduacao_2017_2018.csv'
disciplinas = pd.read_csv(catalogo, sep="|")
disciplinas.head()

,SIGLA,DISCIPLINA,T,P,I,RECOMENDAÇÃO
0,ESHR022-14,Abordagens Tradicionais das Relações Internaci...,4,0,4,Não há
1,ESZM035-17,Aditivação de Polímeros,4,0,4,Síntese de Polímeros; Materiais Poliméricos
2,ESZP041-14,Administração Pública e Reforma do Estado em P...,4,0,4,Não há
3,ESTS016-17,Aerodinâmica I,4,0,5,Dinâmica de Gases
4,ESZS019-17,Aerodinâmica II,4,0,5,Aerodinâmica I


Realizamos um pré-processamento das *strings* desse novo conjunto.

In [0]:
#Remoção de acentos
disciplinas['DISCIPLINA'] = disciplinas['DISCIPLINA'].str.normalize('NFKD')

#Para minusculas
disciplinas['DISCIPLINA'] = disciplinas['DISCIPLINA'].str.lower()

#Encondificação para ASCII ignorando erros
disciplinas['DISCIPLINA'] = disciplinas['DISCIPLINA'].str.encode('ascii', errors='ignore')

#Decoficação para UTF-8, formato mais empregado nessa categoria de análise
disciplinas['DISCIPLINA'] = disciplinas['DISCIPLINA'].str.decode('utf-8')

Antes disso, realizamos uma análise das disciplinas listadas verificando se não há disciplinas com 
nomes replicados.

In [10]:
abs(len(disciplinas['DISCIPLINA'].unique()) - len(disciplinas['DISCIPLINA']))

2

Há duas disciplinas com nomes replicados, sendo esses:

In [11]:
disciplinas[disciplinas['DISCIPLINA'].duplicated()]

,SIGLA,DISCIPLINA,T,P,I,RECOMENDAÇÃO
162,ESZG023-17,contabilidade para engenharia,4,0,5,Economia de Empresas
467,ESHT009-17,historia da cidade e do urbanismo,4,0,4,Não há


Talvez seja interessante reportar à universidade. Seguindo nossa análise, verificamos que há 139 disciplinas que estão nos históricos dos alunos, mas não estão no catálogo de disciplina. Isso ocorre, pois, as disciplinas foram substituidas por outras.

In [12]:
disci_fora_catalogo = tratado['NOME_DISCIPLINA'].unique()[~np.isin(tratado['NOME_DISCIPLINA'].unique(),disciplinas['DISCIPLINA'].unique())]
len(disci_fora_catalogo)

139

Analisando o conjunto de disciplinas elencadas observamos que essas disciplinas são datadas dos primeiros projetos pedagógicos da universidade.

Dessa forma, recorremos ao primeiro catálogo de disciplinas (2011) e analisamos se haverá redução do número de disciplinas sem informações sobre T-P-I.



In [13]:
catalogo2011 = "https://raw.githubusercontent.com/bruAristimunha/miningAbandonment/master/data/catalogo_de_disciplinas_2011.csv"

disciplinas2011 = pd.read_csv(catalogo2011, sep="|")
disciplinas2011.columns = ['SIGLA', 'DISCIPLINA', 'T', 'P', 'I']

#Remoção de acentos
disciplinas2011['DISCIPLINA'] = disciplinas2011['DISCIPLINA'].str.normalize(
                                                                        'NFKD')
      
#Para minusculas
disciplinas2011['DISCIPLINA'] = disciplinas2011['DISCIPLINA'].str.lower()

#Encondificação para ASCII ignorando erros
disciplinas2011['DISCIPLINA'] = disciplinas2011['DISCIPLINA'].str.encode('ascii', 
                                                                errors='ignore')

#Decoficação para UTF-8, formato mais empregado nessa categoria de análise
disciplinas2011['DISCIPLINA'] = disciplinas2011['DISCIPLINA'].str.decode('utf-8')


disciplinas2011.columns

Index(['SIGLA', 'DISCIPLINA', 'T', 'P', 'I'], dtype='object')

In [14]:
disciplinas = disciplinas.append(disciplinas2011, sort=True)


disciplinas =  disciplinas[['SIGLA', 'DISCIPLINA', 'T', 'P', 'I', 'RECOMENDAÇÃO']]
disciplinas.head()

,SIGLA,DISCIPLINA,T,P,I,RECOMENDAÇÃO
0,ESHR022-14,abordagens tradicionais das relacoes internaci...,4,0,4,Não há
1,ESZM035-17,aditivacao de polimeros,4,0,4,Síntese de Polímeros; Materiais Poliméricos
2,ESZP041-14,administracao publica e reforma do estado em p...,4,0,4,Não há
3,ESTS016-17,aerodinamica i,4,0,5,Dinâmica de Gases
4,ESZS019-17,aerodinamica ii,4,0,5,Aerodinâmica I


Com essa abordagem reduzimos para 18 o número de disciplinas com informações faltantes

In [15]:
foraCatalogo = tratado['NOME_DISCIPLINA'].unique()[~np.isin(tratado['NOME_DISCIPLINA'].unique(),disciplinas['DISCIPLINA'].unique())]
print(len(foraCatalogo))

2


Optamos pelo preenchimento manual e adição ao catalogo mais antigo (2011).

In [16]:
foraCatalogo

array(['projeto de aeronavesi', 'combustao'], dtype=object)

Essas disciplinas não foram encontradas em nenhum fonte oficial de informação. Houve apenas disciplinas Combutão I, mas como não há mais informações optamos por ignorar essas instâncias.

Re-ordenamos a nossa matriz com os valores tratados.


In [0]:
tratado.columns = ['ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA',
       'COD_DISCIPLINA', 'CATEGORIA', 'A', 'B', 'C', 'D', 'F', 'O', 'ANO_QUAD',
       'QUAD_CURSADO', 'ANOS_NA_UNIVER']

In [18]:
tratado.head()

,ID DO ALUNO,ANO,QUADRIMESTRE,DISCIPLINA,COD_DISCIPLINA,CATEGORIA,A,B,C,D,F,O,ANO_QUAD,QUAD_CURSADO,ANOS_NA_UNIVER
0,1.0,2011.0,2º Quadrimestre,bases computacionais da ciencia,BIM0005-13,Obrigatória,0.0,0.0,0.0,1.0,0.0,0.0,2011.02º Quadrimestre,1,0
1,1.0,2011.0,2º Quadrimestre,bases matematicas,BIN0003-13,Obrigatória,0.0,0.0,0.0,0.0,1.0,0.0,2011.02º Quadrimestre,1,0
2,1.0,2011.0,2º Quadrimestre,iniciacao a pesquisa cientifica i,ESZX129-13,Opção Limitada,1.0,0.0,0.0,0.0,0.0,0.0,2011.02º Quadrimestre,1,0
3,1.0,2011.0,2º Quadrimestre,origem da vida e diversidade dos seres vivos,BIL0304-13,Obrigatória,0.0,1.0,0.0,0.0,0.0,0.0,2011.02º Quadrimestre,1,0
4,1.0,2011.0,3º Quadrimestre,geometria analitica,BCN0404-13,Obrigatória,0.0,0.0,0.0,0.0,1.0,0.0,2011.03º Quadrimestre,2,0


In [0]:
tratado_01 = tratado.copy()
disciplinas_unique = disciplinas.drop_duplicates('DISCIPLINA')
tratado_02 = tratado_01.merge(disciplinas_unique,on='DISCIPLINA').sort_values(['ID DO ALUNO','ANO'])

In [0]:
tratado_02['CRED'] = tratado_02['T'] + tratado_02['P']

In [0]:
creditos_por_quad = tratado_02.groupby(['ID DO ALUNO', 'ANO', 'QUADRIMESTRE'])['CRED'].apply(sum)

In [22]:
tratado_02.columns

Index(['ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA', 'COD_DISCIPLINA',
       'CATEGORIA', 'A', 'B', 'C', 'D', 'F', 'O', 'ANO_QUAD', 'QUAD_CURSADO',
       'ANOS_NA_UNIVER', 'SIGLA', 'T', 'P', 'I', 'RECOMENDAÇÃO', 'CRED'],
      dtype='object')

## 3. Coeficiente de Rendimento por quadrimestre


In [0]:
tratado_02['A_'] = tratado_02['A']*4
tratado_02['B_'] = tratado_02['B']*3
tratado_02['C_'] = tratado_02['C']*2
tratado_02['D_'] = tratado_02['D']*1
tratado_02['F_'] = tratado_02['F']*0
tratado_02['O_'] = tratado_02['O']*0

tratado_02['CONCEITO'] = tratado_02['A_']+tratado_02['B_']+tratado_02['C_']+tratado_02['D_']+tratado_02['F_']+tratado_02['O_']
          
          
tratado_02['CONCEITO_CRED'] = tratado_02['CONCEITO']*tratado_02['CRED']

cr_por_quadrimestre = tratado_02.groupby([
  'ID DO ALUNO','ANO','QUADRIMESTRE'])['CONCEITO_CRED'].apply(sum)/tratado_02.groupby([
  'ID DO ALUNO','ANO','QUADRIMESTRE'])['CRED'].apply(sum)

In [24]:
cr_por_quadrimestre

ID DO ALUNO  ANO     QUADRIMESTRE   
1.0          2011.0  2º Quadrimestre    1.727273
                     3º Quadrimestre    1.666667
             2012.0  1º Quadrimestre    2.176471
                     2º Quadrimestre    3.625000
                     3º Quadrimestre    3.000000
             2013.0  1º Quadrimestre    3.000000
                     2º Quadrimestre    3.500000
                     3º Quadrimestre    3.000000
             2015.0  3º Quadrimestre    3.000000
             2016.0  1º Quadrimestre    2.428571
                     2º Quadrimestre    1.846154
                     3º Quadrimestre    2.666667
             2017.0  1º Quadrimestre    0.600000
                     2º Quadrimestre    3.384615
                     3º Quadrimestre    3.200000
2.0          2013.0  1º Quadrimestre    3.000000
                     2º Quadrimestre    2.000000
                     3º Quadrimestre    1.909091
             2014.0  1º Quadrimestre    2.000000
                     2º Quadrime

## 4. Coeficiente de Rendimento por ano


In [25]:
cr_por_ano = tratado_02.groupby([
    'ID DO ALUNO', 'ANO'])['CONCEITO_CRED'].apply(sum)/tratado_02.groupby([
    'ID DO ALUNO','ANO'])['CRED'].apply(sum)

cr_por_ano.head()

ID DO ALUNO  ANO   
1.0          2011.0    1.700000
             2012.0    2.742857
             2013.0    3.181818
             2015.0    3.000000
             2016.0    2.192308
dtype: float64

## 5. Coeficiente de rendimento acumulado

In [26]:
cr_acumulado = tratado_02.groupby([
    'ID DO ALUNO'])['CONCEITO_CRED'].apply(sum)/tratado_02.groupby([
    'ID DO ALUNO'])['CRED'].apply(sum)
cr_acumulado.head()

ID DO ALUNO
1.0    2.549669
2.0    1.753247
3.0    2.528169
4.0    1.680135
5.0    1.694545
dtype: float64

## 6.  Porcentagem de reprovação na disciplina

In [0]:
tratado_02['APROVADO'] = tratado_02['A']+tratado_02['B']+tratado_02['C']+tratado_02['D']
tratado_02['REPROVADO'] = tratado_02['F']
tratado_02['ABANDONO'] = tratado_02['O']

porcent_aprov = tratado_02.groupby(['DISCIPLINA'])['APROVADO'].apply(sum)/(
                      tratado_02.groupby(['DISCIPLINA'])['APROVADO'].apply(sum)+
                      tratado_02.groupby(['DISCIPLINA'])['REPROVADO'].apply(sum)+
                      tratado_02.groupby(['DISCIPLINA'])['ABANDONO'].apply(sum))

porcent_reprov = tratado_02.groupby(['DISCIPLINA'])['REPROVADO'].apply(sum)/(
                      tratado_02.groupby(['DISCIPLINA'])['APROVADO'].apply(sum)+
                      tratado_02.groupby(['DISCIPLINA'])['REPROVADO'].apply(sum)+
                      tratado_02.groupby(['DISCIPLINA'])['ABANDONO'].apply(sum))

porcent_abando = tratado_02.groupby(['DISCIPLINA'])['ABANDONO'].apply(sum)/(
                      tratado_02.groupby(['DISCIPLINA'])['APROVADO'].apply(sum)+
                      tratado_02.groupby(['DISCIPLINA'])['REPROVADO'].apply(sum)+
                      tratado_02.groupby(['DISCIPLINA'])['ABANDONO'].apply(sum))


In [28]:
porcent_aprov.head()

DISCIPLINA
acionamentos eletricos                             0.909091
acumuladores de energia                            0.750000
administracao municipal e desenvolvimento local    1.000000
aerodinamica i                                     0.857143
aeroelasticidade                                   0.750000
dtype: float64

In [29]:
porcent_reprov.head()

DISCIPLINA
acionamentos eletricos                             0.000000
acumuladores de energia                            0.000000
administracao municipal e desenvolvimento local    0.000000
aerodinamica i                                     0.071429
aeroelasticidade                                   0.250000
dtype: float64

In [30]:
porcent_abando.head()

DISCIPLINA
acionamentos eletricos                             0.090909
acumuladores de energia                            0.250000
administracao municipal e desenvolvimento local    0.000000
aerodinamica i                                     0.071429
aeroelasticidade                                   0.000000
dtype: float64

## 7  Diferença por quadrimestre da média de créditos global menos a quantidade no quadrimestre


In [31]:
quantCredPorQuadrimestre = tratado_02.groupby(['ID DO ALUNO','ANO','QUADRIMESTRE'])['CRED'].apply(sum)

quantCredPorQuadrimestre.head()

ID DO ALUNO  ANO     QUADRIMESTRE   
1.0          2011.0  2º Quadrimestre    11
                     3º Quadrimestre     9
             2012.0  1º Quadrimestre    17
                     2º Quadrimestre     8
                     3º Quadrimestre    10
Name: CRED, dtype: int64

In [32]:
media = quantCredPorQuadrimestre.reset_index().groupby(['ID DO ALUNO'])['CRED'].apply(np.mean)
media.head()

ID DO ALUNO
1.0    10.066667
2.0    10.266667
3.0    10.142857
4.0    10.607143
5.0    10.576923
Name: CRED, dtype: float64

In [0]:
diferencaMedia = quantCredPorQuadrimestre - media

## 8.  z-score entre a quantidade créditos cursadas;  


In [34]:
std = quantCredPorQuadrimestre.reset_index().groupby(['ID DO ALUNO'])['CRED'].apply(np.std)

z_score_creditos = (quantCredPorQuadrimestre - media)/std

z_score_creditos.head()

#Conferencia com a implementação oficial:
#quantCredPorQuadrimestre.reset_index().groupby(['ID DO ALUNO'])['CRED'].apply(sci.stats.zscore)
#bate, mas formatação estranha.

ID DO ALUNO  ANO     QUADRIMESTRE   
1.0          2011.0  2º Quadrimestre    0.283190
                     3º Quadrimestre   -0.323645
             2012.0  1º Quadrimestre    2.103695
                     2º Quadrimestre   -0.627063
                     3º Quadrimestre   -0.020228
Name: CRED, dtype: float64

# Juntando em um único dataset

In [35]:
tratado_02.columns



Index(['ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA', 'COD_DISCIPLINA',
       'CATEGORIA', 'A', 'B', 'C', 'D', 'F', 'O', 'ANO_QUAD', 'QUAD_CURSADO',
       'ANOS_NA_UNIVER', 'SIGLA', 'T', 'P', 'I', 'RECOMENDAÇÃO', 'CRED', 'A_',
       'B_', 'C_', 'D_', 'F_', 'O_', 'CONCEITO', 'CONCEITO_CRED', 'APROVADO',
       'REPROVADO', 'ABANDONO'],
      dtype='object')

In [0]:
porcent_aprov = porcent_aprov.reset_index()

porcent_aprov.columns = ['DISCIPLINA','PORC_APROV']

In [0]:
tratado_02 = tratado_02.merge(porcent_aprov,on='DISCIPLINA').sort_values(['ID DO ALUNO','ANO'])

In [0]:
porcent_reprov = porcent_reprov.reset_index()

porcent_reprov.columns = ['DISCIPLINA','PORC_REPROV']

tratado_02 = tratado_02.merge(porcent_reprov,on='DISCIPLINA').sort_values(['ID DO ALUNO','ANO'])

porcent_abando = porcent_abando.reset_index()

porcent_abando.columns = ['DISCIPLINA','PORC_ABANDO']

tratado_02 = tratado_02.merge(porcent_abando,on='DISCIPLINA').sort_values(['ID DO ALUNO','ANO'])

In [0]:
quantCredPorQuadrimestre = quantCredPorQuadrimestre.reset_index()
quantCredPorQuadrimestre.columns = ['ID DO ALUNO','ANO','QUADRIMESTRE','CRED_POR_QUAD']


In [0]:
tratado_02 = tratado_02.merge(quantCredPorQuadrimestre,on=['ID DO ALUNO','ANO','QUADRIMESTRE']).sort_values(['ID DO ALUNO','ANO','QUADRIMESTRE']) 

In [0]:
diferencaMedia = diferencaMedia.reset_index()
diferencaMedia.columns = ['ID DO ALUNO','ANO','QUADRIMESTRE','DIFERENCA_A_MEDIA']

tratado_02 = tratado_02.merge(diferencaMedia,on=['ID DO ALUNO','ANO','QUADRIMESTRE']).sort_values(['ID DO ALUNO','ANO','QUADRIMESTRE']) 

In [0]:
z_score_creditos = z_score_creditos.reset_index()
z_score_creditos.columns = ['ID DO ALUNO','ANO','QUADRIMESTRE','Z_SCORE']

tratado_02 = tratado_02.merge(z_score_creditos,on=['ID DO ALUNO','ANO','QUADRIMESTRE']).sort_values(['ID DO ALUNO','ANO','QUADRIMESTRE']) 

In [0]:
cr_acumulado = cr_acumulado.reset_index()

cr_acumulado.columns = ['ID DO ALUNO','CR_ACUMULADO']

tratado_02 = tratado_02.merge(cr_acumulado,on=['ID DO ALUNO']).sort_values(['ID DO ALUNO','ANO','QUADRIMESTRE']) 

In [0]:
cr_por_quadrimestre = cr_por_quadrimestre.reset_index()

cr_por_quadrimestre.columns = ['ID DO ALUNO','ANO','QUADRIMESTRE','CR_QUAD']

tratado_02 = tratado_02.merge(cr_por_quadrimestre,on=['ID DO ALUNO','ANO','QUADRIMESTRE']).sort_values(['ID DO ALUNO','ANO','QUADRIMESTRE']) 

In [0]:
cr_por_ano = cr_por_ano.reset_index()

cr_por_ano.columns = ['ID DO ALUNO','ANO','CR_POR_ANO']

tratado_02 = tratado_02.merge(cr_por_ano,on=['ID DO ALUNO','ANO']).sort_values(['ID DO ALUNO','ANO','QUADRIMESTRE']) 



In [0]:
tratado_03 = tratado_02.drop(['A_','B_','C_', 'D_', 'F_', 'O_','RECOMENDAÇÃO'],axis=1)

In [0]:
tratado_03 = tratado_03[['ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA', 
            'COD_DISCIPLINA','CATEGORIA','ANO_QUAD', 'QUAD_CURSADO',
            'ANOS_NA_UNIVER', 'SIGLA', 'T', 'P', 'I', 
            'CRED', 'CONCEITO', 'CONCEITO_CRED',
           'PORC_APROV', 'PORC_REPROV', 'PORC_ABANDO', 'CRED_POR_QUAD',
           'DIFERENCA_A_MEDIA', 'Z_SCORE', 'CR_ACUMULADO', 'CR_QUAD',
           'CR_POR_ANO', 'A', 'B', 'C', 'D', 'F', 'O',
            'APROVADO', 'REPROVADO', 'ABANDONO']]

In [0]:
tratado_03.to_csv("dataset.csv",sep="|")
from google.colab import files


In [0]:
files.download('dataset.csv')